## Requirements
1. Apache Spark binary (https://spark.apache.org/)
2. For Windows: winutils (https://medium.com/@dvainrub/how-to-install-apache-spark-2-x-in-your-pc-e2047246ffc3)
3. Setting ```JAVA_HOME```, ```SPARK_HOME```, and ```HADOOP_HOME```
4. Python 3.x (from Anaconda distribution)
5. ```findspark``` https://pypi.org/project/findspark/
6. Jupyter Notebook (available from Anaconda installation)

### References
https://spark.apache.org/docs/2.3.3/sql-programming-guide.html

## Spark Initialization

In [1]:
# Import findspark to read SPARK_HOME and HADOOP_HOME
import findspark
findspark.init()

In [2]:
# Import required library
from pyspark.sql import SparkSession

# Create Spark Session
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [3]:
# Print Spark object ID
print(spark)

## Loading Data using Spark

In [4]:
df = spark.read.csv("/Users/gunstringer/Downloads/epa_hap_daily_summary.csv", header=True, inferSchema=True)

In [5]:
df.show()

+----------+-----------+--------+--------------+---+---------+-----------+-------+--------------------+---------------+------------------+-------------------+--------------------+----------+-----------------+-------------------+---------------+---------------+--------------+----+-----------+--------------------+--------------------+--------------------+------------+----------------+-------------+--------------------+-------------------+
|state_code|county_code|site_num|parameter_code|poc| latitude|  longitude|  datum|      parameter_name|sample_duration|pollutant_standard|         date_local|    units_of_measure|event_type|observation_count|observation_percent|arithmetic_mean|first_max_value|first_max_hour| aqi|method_code|         method_name|     local_site_name|             address|  state_name|     county_name|    city_name|           cbsa_name|date_of_last_change|
+----------+-----------+--------+--------------+---+---------+-----------+-------+--------------------+---------------

In [126]:
df.printSchema()

root
 |-- state_code: integer (nullable = true)
 |-- county_code: integer (nullable = true)
 |-- site_num: integer (nullable = true)
 |-- parameter_code: integer (nullable = true)
 |-- poc: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- datum: string (nullable = true)
 |-- parameter_name: string (nullable = true)
 |-- sample_duration: string (nullable = true)
 |-- pollutant_standard: string (nullable = true)
 |-- date_local: timestamp (nullable = true)
 |-- units_of_measure: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- observation_count: integer (nullable = true)
 |-- observation_percent: double (nullable = true)
 |-- arithmetic_mean: double (nullable = true)
 |-- first_max_value: double (nullable = true)
 |-- first_max_hour: integer (nullable = true)
 |-- aqi: string (nullable = true)
 |-- method_code: integer (nullable = true)
 |-- method_name: string (nullable = true)
 |-- local_site_name: st

In [8]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("pollution")

In [29]:
row_count = spark.sql ("SELECT COUNT(1)FROM pollution")

In [30]:
row_count.show()

+--------+
|count(1)|
+--------+
| 8097069|
+--------+



In [9]:
result = spark.sql("SELECT DISTINCT parameter_name FROM pollution")

In [10]:
result.show()

+--------------------+
|      parameter_name|
+--------------------+
|             Benzene|
|Chromium VI (TSP) LC|
|  12-Dichloropropane|
|Acrolein - Unveri...|
| Tetrachloroethylene|
|       Lead PM10 STP|
|     Nickel PM10 STP|
|    Nickel (TSP) STP|
|   Chromium PM10 STP|
|  Ethylene dibromide|
|    Mercury PM10 STP|
|  Chromium (TSP) STP|
|cis-13-Dichloropr...|
|      Vinyl chloride|
|   Trichloroethylene|
|       Lead PM2.5 LC|
|          Chloroform|
|  Manganese PM10 STP|
|        Acetaldehyde|
|  Beryllium PM2.5 LC|
+--------------------+
only showing top 20 rows



## Data Mining Process

In [65]:
# Mencari rata rata konsentrasi air polutant terbesar dalam suatu state (ex: pennsylvania)

query1 = spark.sql ("SELECT parameter_name, AVG(arithmetic_mean),units_of_measure FROM pollution\
                        WHERE state_name = 'Pennsylvania' \
                        GROUP BY parameter_name, units_of_measure \
                        ORDER BY AVG(arithmetic_mean) DESC")

In [66]:
query1.show()

+--------------------+--------------------+--------------------+
|      parameter_name|avg(arithmetic_mean)|    units_of_measure|
+--------------------+--------------------+--------------------+
|        Formaldehyde|  2.6133003936669663|Parts per billion...|
|             Benzene|  2.2339411705663816|Parts per billion...|
|        Acetaldehyde|  2.0236999796918775|Parts per billion...|
|   Trichloroethylene| 0.24696496444731733|Parts per billion...|
|     Dichloromethane|  0.2274077303487766|Parts per billion...|
|Acrolein - Unveri...| 0.22246105104761915|Parts per billion...|
| Tetrachloroethylene| 0.20841215255332896|Parts per billion...|
|        13-Butadiene| 0.20047134502923972|Parts per billion...|
|Carbon tetrachloride| 0.09884115055713964|Parts per billion...|
|          Chloroform| 0.03560626050963646|Parts per billion...|
| Beryllium (TSP) STP| 0.02544813278008298|Nanograms/cubic m...|
| Manganese (TSP) STP|0.025297648530331437|Micrograms/cubic ...|
| Ethylene dichloride|0.0

In [75]:
# state yang memiliki rata rata suatu air polutant (ex: Benzene) paling banyak

query2 = spark.sql ("SELECT state_name, AVG(arithmetic_mean), units_of_measure FROM pollution\
                        WHERE parameter_name = 'Benzene'\
                        GROUP BY state_name, units_of_measure\
                        ORDER BY AVG(arithmetic_mean) DESC")

In [76]:
query2.show()

+-----------------+--------------------+--------------------+
|       state_name|avg(arithmetic_mean)|    units_of_measure|
+-----------------+--------------------+--------------------+
|Country Of Mexico|   8.708797468354431|Parts per billion...|
|          Alabama|   6.430967802146523|Parts per billion...|
|           Kansas|   5.406358490566038|Parts per billion...|
|           Alaska|   4.649691056910569|Parts per billion...|
|       New Mexico|  4.2419974639498434|Parts per billion...|
|         Missouri|  4.0470664072806475|Parts per billion...|
|         Colorado|   3.530716491661107|Parts per billion...|
|          Arizona|  3.2813660599781884|Parts per billion...|
|            Texas|   3.165615802749503|Parts per billion...|
|         Arkansas|  3.1010606060606065|Parts per billion...|
|             Utah|   3.088827945776851|Parts per billion...|
|      Puerto Rico|   3.016304182509505|Parts per billion...|
|       California|  2.9796335305933703|Parts per billion...|
|       

In [119]:
# nilai rata-rata konsentrasi zat polutan dari tahun ke tahun pada US

query3 = spark.sql ("SELECT EXTRACT (YEAR FROM date_local), parameter_name, AVG(arithmetic_mean), units_of_measure FROM pollution\
                    GROUP BY EXTRACT(YEAR FROM date_local), units_of_measure, parameter_name \
                    ORDER BY EXTRACT(YEAR FROM date_local) DESC")

In [120]:
query3.show()

+------------------------------+--------------------+--------------------+--------------------+
|year(CAST(date_local AS DATE))|      parameter_name|avg(arithmetic_mean)|    units_of_measure|
+------------------------------+--------------------+--------------------+--------------------+
|                          2017|   Trichloroethylene|0.003557620817843...|Parts per billion...|
|                          2017|  Chromium (TSP) STP|0.006750087719298247|Micrograms/cubic ...|
|                          2017|        13-Butadiene| 0.10028373702422147|Parts per billion...|
|                          2017|      Vinyl chloride|                 0.0|Parts per billion...|
|                          2017|     Dichloromethane|  0.6513717472118958|Parts per billion...|
|                          2017|             Benzene|  1.6149861979010498|Parts per billion...|
|                          2017|1122-Tetrachloroe...|            3.125E-4|Parts per billion...|
|                          2017|trans-13

In [121]:
# nilai konsentrasi benzene dari tahun ke tahun pada US

query4 = spark.sql ("SELECT EXTRACT (YEAR FROM date_local), AVG(arithmetic_mean), units_of_measure FROM pollution\
                    WHERE parameter_name = 'Benzene' \
                    GROUP BY EXTRACT(YEAR FROM date_local), units_of_measure \
                    ORDER BY EXTRACT(YEAR FROM date_local) DESC")

In [122]:
query4.show()

+------------------------------+--------------------+--------------------+
|year(CAST(date_local AS DATE))|avg(arithmetic_mean)|    units_of_measure|
+------------------------------+--------------------+--------------------+
|                          2017|  1.6149861979010498|Parts per billion...|
|                          2016|  1.2660918363206974|Parts per billion...|
|                          2015|  1.3288374331274566|Parts per billion...|
|                          2014|   1.324096420975431|Parts per billion...|
|                          2013|   1.451811187132259|Parts per billion...|
|                          2012|   1.603766487430922|Parts per billion...|
|                          2011|  1.5579733075711695|Parts per billion...|
|                          2010|  1.7227266005456587|Parts per billion...|
|                          2009|   1.894231711105646|Parts per billion...|
|                          2008|  2.3998395963532064|Parts per billion...|
|                        

In [130]:
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd

data = [go.Histogram(x=query4.toPandas()['d1'])]
py.iplot(data, filename="spark/less_2_hour_rides")

Py4JJavaError: An error occurred while calling o1753.collectToPython.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange rangepartitioning(year(CAST(date_local AS DATE))#930 DESC NULLS LAST, 200)
+- *(2) HashAggregate(keys=[year(cast(date_local#21 as date))#954, units_of_measure#22], functions=[avg(arithmetic_mean#26)], output=[year(CAST(date_local AS DATE))#930, avg(arithmetic_mean)#929, units_of_measure#22])
   +- Exchange hashpartitioning(year(cast(date_local#21 as date))#954, units_of_measure#22, 200)
      +- *(1) HashAggregate(keys=[year(cast(date_local#21 as date)) AS year(cast(date_local#21 as date))#954, units_of_measure#22], functions=[partial_avg(arithmetic_mean#26)], output=[year(cast(date_local#21 as date))#954, units_of_measure#22, sum#947, count#948L])
         +- *(1) Project [date_local#21, units_of_measure#22, arithmetic_mean#26]
            +- *(1) Filter (isnotnull(parameter_name#18) && (parameter_name#18 = Benzene))
          